In [1]:
import pandas as pd
import uproot
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import crystalball
from scipy.optimize import curve_fit,Bounds
from scipy.integrate import simps
import zfit_physics as zphysics
import zfit
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.pyplot as plt
import mplhep
import uncertainties 
from uncertainties import ufloat
import tensorflow as tf

plt.style.use('belle2')
#%jsroot on


%run /afs/desy.de/user/j/jipatel/MT/MC_fit/Files_After_Acceptance/Signal_Background_thesis_final/Function.ipynb

2024-09-03 19:47:13.179447: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-03 19:47:13.214768: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-03 19:47:13.215842: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-03 19:47:14.446656: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/afs/desy.de/user/j/jipatel/.local/lib/python3.8/site-packages/zfit/__init__.py:63: UserWarning: TensorFlow warnings are by default suppressed by zfit. In order to show them, set the environment variable ZFIT_DISABLE_TF_WARNINGS=0. In order to suppress the TensorFlow warnings AND this warnin

## 200

In [2]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_mu_signal, mc_mu_only_signal, mc_mu_scf, mc_mu_bkg, mc_mu_signal_weight, mc_mu_only_signal_weight, mc_mu_scf_weight, mc_mu_bkg_weight = process_MC_data_combined(
        file_signal_mu_Bppip,file_bkg_mu_Bppip, 'mu', '521', 'pip', 'mumu', 3, 'rec521', 'weights', 200
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'mu', 'signal', mc_mu_only_signal["Mbc"],  'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'signal', mc_mu_only_signal["deltaE"], 'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'scf', mc_mu_scf["Mbc"],  'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'scf', mc_mu_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'bkg', mc_mu_bkg["Mbc"],  'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'bkg', mc_mu_bkg["deltaE"], 'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_mu_only_signal["Mbc"]
    Signal_deltaE = mc_mu_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_mu_scf["Mbc"], mc_mu_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_mu_scf["deltaE"], mc_mu_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)


    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])

# Save the DataFrame to a CSV file
signal_yields_df.to_csv('Bppip_q2reg3_decay_mumu_signal_yields_and_errors_200.csv', index=False)

print("Signal yields and errors saved to 'Bppip_q2reg3_decay_mumu_signal_yields_and_errors_200.csv'")


Running iteration 1/300...
200
538


/afs/desy.de/user/j/jipatel/.local/lib/python3.8/site-packages/zfit/core/basepdf.py:585: UserWarning: As `copy` is not yet properly implemented, this may fails (for ProductPDF for example?). Thiswill be fixed in the future.
  warnings.warn(
/afs/desy.de/user/j/jipatel/.local/lib/python3.8/site-packages/zfit/minimizers/fitresult.py:1340: FutureWarning: 'minuit_minos' will be changed as the default errors method to a custom implementationwith the same functionality. If you want to make sure that 'minuit_minos' will be used in the future, add it explicitly as in `errors(method='minuit_minos')`
  warnings.warn(


name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.1636                 232.373  -     16   +     18       False
background_yield_0.1636             505.382  -     23   +     24       False
frac_Mbc_bkg1_0.1636               0.426048  -  0.053   +  0.052       False
frac_deltaE_bkg1_0.1636           -0.102106  -   0.14   +   0.14       False
True
Iteration 1 - Signal Yield: 232.3731790415627 ± 17.048225500302372
Running iteration 2/300...
200
538
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.9448                 221.053  -     16   +     17       False
background_yield_0.9448             516.711  -     24   +     24       False
frac_Mbc_bkg1_0.9448               0.454212  -  0.053   +  0.052       False
frac_deltaE_bkg1_0.9448          -0.0842711  -

## 150

In [5]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_mu_signal, mc_mu_only_signal, mc_mu_scf, mc_mu_bkg, mc_mu_signal_weight, mc_mu_only_signal_weight, mc_mu_scf_weight, mc_mu_bkg_weight = process_MC_data_combined(
        file_signal_mu_Bppip,file_bkg_mu_Bppip, 'mu', '521', 'pip', 'mumu', 3, 'rec521', 'weights', 150
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'mu', 'signal', mc_mu_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'signal', mc_mu_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'scf', mc_mu_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'scf', mc_mu_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'bkg', mc_mu_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'bkg', mc_mu_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_mu_only_signal["Mbc"]
    Signal_deltaE = mc_mu_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_mu_scf["Mbc"], mc_mu_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_mu_scf["deltaE"], mc_mu_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])

# Save the DataFrame to a CSV file
signal_yields_df.to_csv('Bppip_q2reg3_decay_mumu_signal_yields_and_errors_150.csv', index=False)

print("Signal yields and errors saved to 'Bppip_q2reg3_decay_mumu_signal_yields_and_errors_150.csv'")


Running iteration 1/300...
150
538
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.2021                 176.276  -     15   +     16       False
background_yield_0.2021             511.641  -     24   +     24       False
frac_Mbc_bkg1_0.2021                0.44303  -  0.053   +  0.052       False
frac_deltaE_bkg1_0.2021          -0.0805747  -   0.14   +   0.14       False
True
Iteration 1 - Signal Yield: 176.2758012197334 ± 15.252410909033353
Running iteration 2/300...
150
538
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.0689                 169.519  -     15   +     16       False
background_yield_0.0689             518.409  -     24   +     24       False
frac_Mbc_bkg1_0.0689               0.458097  -  0.052   +  0.051       False
frac_deltaE

## 100

In [6]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_mu_signal, mc_mu_only_signal, mc_mu_scf, mc_mu_bkg, mc_mu_signal_weight, mc_mu_only_signal_weight, mc_mu_scf_weight, mc_mu_bkg_weight = process_MC_data_combined(
        file_signal_mu_Bppip,file_bkg_mu_Bppip, 'mu', '521', 'pip', 'mumu', 3, 'rec521', 'weights', 100
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'mu', 'signal', mc_mu_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'signal', mc_mu_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'scf', mc_mu_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'scf', mc_mu_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'bkg', mc_mu_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'bkg', mc_mu_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_mu_only_signal["Mbc"]
    Signal_deltaE = mc_mu_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_mu_scf["Mbc"], mc_mu_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_mu_scf["deltaE"], mc_mu_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('Bppip_q2reg3_decay_mumu_signal_yields_and_errors_100.csv', index=False)

print("Signal yields and errors saved to 'Bppip_q2reg3_decay_mumu_signal_yields_and_errors_100.csv'")


Running iteration 1/300...
100
538
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.4817                  127.14  -     13   +     13       False
background_yield_0.4817             511.618  -     24   +     23       False
frac_Mbc_bkg1_0.4817               0.440027  -  0.051   +  0.052       False
frac_deltaE_bkg1_0.4817          -0.0800543  -   0.13   +   0.14       False
True
Iteration 1 - Signal Yield: 127.139653190125 ± 13.251066269146316
Running iteration 2/300...
100
538
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.9183                 123.602  -     13   +     13       False
background_yield_0.9183             515.167  -     24   +     24       False
frac_Mbc_bkg1_0.9183               0.446642  -  0.051   +  0.052       False
frac_deltaE_

## 75

In [7]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_mu_signal, mc_mu_only_signal, mc_mu_scf, mc_mu_bkg, mc_mu_signal_weight, mc_mu_only_signal_weight, mc_mu_scf_weight, mc_mu_bkg_weight = process_MC_data_combined(
        file_signal_mu_Bppip,file_bkg_mu_Bppip, 'mu', '521', 'pip', 'mumu', 3, 'rec521', 'weights', 75
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'mu', 'signal', mc_mu_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'signal', mc_mu_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'scf', mc_mu_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'scf', mc_mu_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'bkg', mc_mu_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'bkg', mc_mu_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_mu_only_signal["Mbc"]
    Signal_deltaE = mc_mu_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_mu_scf["Mbc"], mc_mu_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_mu_scf["deltaE"], mc_mu_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('Bppip_q2reg3_decay_mumu_signal_yields_and_errors_75.csv', index=False)

print("Signal yields and errors saved to 'Bppip_q2reg3_decay_mumu_signal_yields_and_errors_75.csv'")


Running iteration 1/300...
75
538
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.5176                 103.047  -     12   +     13       False
background_yield_0.5176             509.738  -     23   +     24       False
frac_Mbc_bkg1_0.5176               0.436729  -  0.051   +  0.052       False
frac_deltaE_bkg1_0.5176          -0.0868603  -   0.13   +   0.14       False
True
Iteration 1 - Signal Yield: 103.04699396072593 ± 12.148420577832244
Running iteration 2/300...
75
538
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.6967                 98.7812  -     11   +     13       False
background_yield_0.6967             514.059  -     23   +     24       False
frac_Mbc_bkg1_0.6967               0.447863  -  0.052   +  0.051       False
frac_deltaE_

## 50

In [8]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_mu_signal, mc_mu_only_signal, mc_mu_scf, mc_mu_bkg, mc_mu_signal_weight, mc_mu_only_signal_weight, mc_mu_scf_weight, mc_mu_bkg_weight = process_MC_data_combined(
        file_signal_mu_Bppip,file_bkg_mu_Bppip, 'mu', '521', 'pip', 'mumu', 3, 'rec521', 'weights', 50
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'mu', 'signal', mc_mu_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'signal', mc_mu_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'scf', mc_mu_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'scf', mc_mu_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'bkg', mc_mu_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'bkg', mc_mu_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_mu_only_signal["Mbc"]
    Signal_deltaE = mc_mu_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_mu_scf["Mbc"], mc_mu_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_mu_scf["deltaE"], mc_mu_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])

# Save the DataFrame to a CSV file
signal_yields_df.to_csv('Bppip_q2reg3_decay_mumu_signal_yields_and_errors_50.csv', index=False)

print("Signal yields and errors saved to 'Bppip_q2reg3_decay_mumu_signal_yields_and_errors_50.csv'")


Running iteration 1/300...
50
538
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.5239                 75.2996  -     10   +     11       False
background_yield_0.5239             512.408  -     23   +     24       False
frac_Mbc_bkg1_0.5239               0.443719  -  0.051   +  0.051       False
frac_deltaE_bkg1_0.5239          -0.0548034  -   0.13   +   0.14       False
True
Iteration 1 - Signal Yield: 75.29955968269422 ± 10.829743514716474
Running iteration 2/300...
50
538
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.8955                 74.3762  -     10   +     11       False
background_yield_0.8955             513.567  -     23   +     24       False
frac_Mbc_bkg1_0.8955               0.446904  -  0.051   +  0.051       False
frac_deltaE_b

## 25

In [9]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_mu_signal, mc_mu_only_signal, mc_mu_scf, mc_mu_bkg, mc_mu_signal_weight, mc_mu_only_signal_weight, mc_mu_scf_weight, mc_mu_bkg_weight = process_MC_data_combined(
        file_signal_mu_Bppip,file_bkg_mu_Bppip, 'mu', '521', 'pip', 'mumu', 3, 'rec521', 'weights', 25
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'mu', 'signal', mc_mu_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'signal', mc_mu_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'scf', mc_mu_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'scf', mc_mu_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'bkg', mc_mu_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'bkg', mc_mu_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_mu_only_signal["Mbc"]
    Signal_deltaE = mc_mu_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_mu_scf["Mbc"], mc_mu_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_mu_scf["deltaE"], mc_mu_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('Bppip_q2reg3_decay_mumu_signal_yields_and_errors_25.csv', index=False)

print("Signal yields and errors saved to 'Bppip_q2reg3_decay_mumu_signal_yields_and_errors_25.csv'")


Running iteration 1/300...
25
538
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.0356                  49.136  -      9   +    9.5       False
background_yield_0.0356             513.992  -     23   +     24       False
frac_Mbc_bkg1_0.0356               0.447614  -   0.05   +  0.051       False
frac_deltaE_bkg1_0.0356          -0.0552011  -   0.13   +   0.14       False
True
Iteration 1 - Signal Yield: 49.1359491047435 ± 9.265324700262855
Running iteration 2/300...
25
538
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.2012                 48.1414  -    9.2   +    9.3       False
background_yield_0.2012             515.217  -     23   +     24       False
frac_Mbc_bkg1_0.2012               0.450001  -   0.05   +  0.051       False
frac_deltaE_bkg

## 10

In [10]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_mu_signal, mc_mu_only_signal, mc_mu_scf, mc_mu_bkg, mc_mu_signal_weight, mc_mu_only_signal_weight, mc_mu_scf_weight, mc_mu_bkg_weight = process_MC_data_combined(
        file_signal_mu_Bppip,file_bkg_mu_Bppip, 'mu', '521', 'pip', 'mumu', 3, 'rec521', 'weights', 10
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'mu', 'signal', mc_mu_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'signal', mc_mu_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'scf', mc_mu_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'scf', mc_mu_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'bkg', mc_mu_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'bkg', mc_mu_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_mu_only_signal["Mbc"]
    Signal_deltaE = mc_mu_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_mu_scf["Mbc"], mc_mu_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_mu_scf["deltaE"], mc_mu_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('Bppip_q2reg3_decay_mumu_signal_yields_and_errors_10.csv', index=False)

print("Signal yields and errors saved to 'Bppip_q2reg3_decay_mumu_signal_yields_and_errors_10.csv'")


Running iteration 1/300...
10
538
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.4607                 32.9767  -      8   +    8.5       False
background_yield_0.4607             514.567  -     23   +     24       False
frac_Mbc_bkg1_0.4607               0.450079  -   0.05   +  0.051       False
frac_deltaE_bkg1_0.4607          -0.0481821  -   0.13   +   0.14       False
True
Iteration 1 - Signal Yield: 32.97672216964156 ± 8.235013321634009
Running iteration 2/300...
10
538
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.6598                 32.7643  -    7.9   +    8.6       False
background_yield_0.6598              515.06  -     23   +     24       False
frac_Mbc_bkg1_0.6598               0.451706  -   0.05   +   0.05       False
frac_deltaE_bk

## 5

In [11]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_mu_signal, mc_mu_only_signal, mc_mu_scf, mc_mu_bkg, mc_mu_signal_weight, mc_mu_only_signal_weight, mc_mu_scf_weight, mc_mu_bkg_weight = process_MC_data_combined(
        file_signal_mu_Bppip,file_bkg_mu_Bppip, 'mu', '521', 'pip', 'mumu', 3, 'rec521', 'weights', 5
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'mu', 'signal', mc_mu_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'signal', mc_mu_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'scf', mc_mu_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'scf', mc_mu_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'bkg', mc_mu_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'bkg', mc_mu_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_mu_only_signal["Mbc"]
    Signal_deltaE = mc_mu_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_mu_scf["Mbc"], mc_mu_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_mu_scf["deltaE"], mc_mu_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('Bppip_q2reg3_decay_mumu_signal_yields_and_errors_5.csv', index=False)

print("Signal yields and errors saved to 'Bppip_q2reg3_decay_mumu_signal_yields_and_errors_5.csv'")


Running iteration 1/300...
5
538
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.9501                 26.5181  -    7.4   +    8.1       False
background_yield_0.9501             516.374  -     23   +     24       False
frac_Mbc_bkg1_0.9501               0.454305  -  0.051   +   0.05       False
frac_deltaE_bkg1_0.9501          -0.0394747  -   0.13   +   0.13       False
True
Iteration 1 - Signal Yield: 26.51812851895867 ± 7.771095115787865
Running iteration 2/300...
5
538
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.6176                 29.0221  -    7.6   +    8.2       False
background_yield_0.6176             513.657  -     23   +     24       False
frac_Mbc_bkg1_0.6176               0.448418  -  0.051   +   0.05       False
frac_deltaE_bkg1

## 2

In [12]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_mu_signal, mc_mu_only_signal, mc_mu_scf, mc_mu_bkg, mc_mu_signal_weight, mc_mu_only_signal_weight, mc_mu_scf_weight, mc_mu_bkg_weight = process_MC_data_combined(
        file_signal_mu_Bppip,file_bkg_mu_Bppip, 'mu', '521', 'pip', 'mumu', 3, 'rec521', 'weights', 2
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'mu', 'signal', mc_mu_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'signal', mc_mu_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'scf', mc_mu_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'scf', mc_mu_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'bkg', mc_mu_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'bkg', mc_mu_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_mu_only_signal["Mbc"]
    Signal_deltaE = mc_mu_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_mu_scf["Mbc"], mc_mu_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_mu_scf["deltaE"], mc_mu_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('Bppip_q2reg3_decay_mumu_signal_yields_and_errors_2.csv', index=False)

print("Signal yields and errors saved to 'Bppip_q2reg3_decay_mumu_signal_yields_and_errors_2.csv'")


Running iteration 1/300...
2
538
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.1501                 23.9837  -    7.1   +      8       False
background_yield_0.1501             515.788  -     23   +     24       False
frac_Mbc_bkg1_0.1501               0.453251  -   0.05   +   0.05       False
frac_deltaE_bkg1_0.1501          -0.0434595  -   0.13   +   0.13       False
True
Iteration 1 - Signal Yield: 23.98369088305234 ± 7.568150085676667
Running iteration 2/300...
2
538
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.6913                 25.7279  -    7.2   +      8       False
background_yield_0.6913             514.212  -     23   +     24       False
frac_Mbc_bkg1_0.6913               0.448381  -   0.05   +   0.05       False
frac_deltaE_bkg1

## 0

In [2]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_mu_signal, mc_mu_only_signal, mc_mu_scf, mc_mu_bkg, mc_mu_signal_weight, mc_mu_only_signal_weight, mc_mu_scf_weight, mc_mu_bkg_weight = process_MC_data_combined(
        file_signal_mu_Bppip,file_bkg_mu_Bppip, 'mu', '521', 'pip', 'mumu', 3, 'rec521', 'weights', 0
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'mu', 'signal', mc_mu_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'signal', mc_mu_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'scf', mc_mu_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'scf', mc_mu_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'bkg', mc_mu_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'bkg', mc_mu_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_mu_only_signal["Mbc"]
    Signal_deltaE = mc_mu_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_mu_scf["Mbc"], mc_mu_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_mu_scf["deltaE"], mc_mu_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('Bppip_q2reg3_decay_mumu_signal_yields_and_errors_0.csv', index=False)

print("Signal yields and errors saved to 'Bppip_q2reg3_decay_mumu_signal_yields_and_errors_0.csv'")


Running iteration 1/300...
0
538


/afs/desy.de/user/j/jipatel/.local/lib/python3.8/site-packages/zfit/core/basepdf.py:585: UserWarning: As `copy` is not yet properly implemented, this may fails (for ProductPDF for example?). Thiswill be fixed in the future.
  warnings.warn(
/afs/desy.de/user/j/jipatel/.local/lib/python3.8/site-packages/zfit/minimizers/fitresult.py:1340: FutureWarning: 'minuit_minos' will be changed as the default errors method to a custom implementationwith the same functionality. If you want to make sure that 'minuit_minos' will be used in the future, add it explicitly as in `errors(method='minuit_minos')`
  warnings.warn(


name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.3453                 22.6863  -    7.1   +    7.8       False
background_yield_0.3453             515.316  -     23   +     24       False
frac_Mbc_bkg1_0.3453               0.451166  -   0.05   +   0.05       False
frac_deltaE_bkg1_0.3453          -0.0434472  -   0.13   +   0.13       False
True
Iteration 1 - Signal Yield: 22.686338088095276 ± 7.432819002111163
Running iteration 2/300...
0
538
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.5407                 22.6863  -    7.1   +    7.8       False
background_yield_0.5407             515.316  -     23   +     24       False
frac_Mbc_bkg1_0.5407               0.451166  -   0.05   +   0.05       False
frac_deltaE_bkg1_0.5407          -0.0434472  -  